In [1]:
import numpy as np
import matplotlib.pyplot as plt
from qmrdmft import Calculator, OptimalCalculator, tools
from pyscf import gto, scf, ao2mo
from numba import prange, jit, njit

#import tools

In [2]:
def expand_matrix(P):
    Paa = P
    Pbb = P
    Pab = np.zeros(P.shape)
    Pba = np.zeros(P.shape)
    PE = np.concatenate((np.concatenate((Paa, Pab), axis=1), np.concatenate((Pba, Pbb), axis=1)), axis=0) 
    return PE

In [3]:
# for jit and prange
@jit(nopython=True, parallel=True)
def ONERDMFT_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in range(0,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [4]:
# for spinorbitals, the Umrigar Hartree term is a Hartree term from which the self interaction has been removed 
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_hartree_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M):
        for b in [b for b in range(0,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += n[a]*n[b]*C[mu,a]*C[nu,a]*C[kappa,b]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    return 1/2.*energy

In [5]:
@jit(nopython=True, parallel=True)
def ONERDMFT_Umrigar_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in [b for b in range(0,M//2) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in [b for b in range(M//2,M) if b!=a ]:
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]

    

    return -1/2.*energy

In [6]:
@jit(parallel=True)
def ONERDMFT_Mueller_exchange_correlation_energy_parallel(Fouridx, C, n):
    energy = 0
    M = C.shape[0]
    K = Fouridx.shape[0]
    for a in  prange(0,M//2):
        for b in range(0,M//2):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]
    for a in  prange(M//2,M):
        for b in range(M//2,M):
            for mu in range(0,M):
                for nu in range(0,M):
                    for kappa in range(0,M):
                        for lamda in range(0,M):
                            energy += np.sqrt(n[a]*n[b])*C[mu,a]*C[nu,b]*C[kappa,a]*C[lamda,b]*Fouridx[mu%K,nu%K,kappa%K,lamda%K]


    return -1/2.*energy

In [19]:
element = "He"
mol = gto.Mole()
mol.atom = f'''{element}  0 0 0; {element} 0 0 1'''
mol.charge = 0
mol.spin = 0
#mol.basis = 'Sto-3g'
mol.basis = "6-31g" 
mol.unit = 'AU'
mol.build()

In [20]:
# get molecuar quantities 
eri = mol.intor('int2e')
S = mol.intor('int1e_ovlp')
N_a, N_b = mol.nelec
M = eri.shape[0]

In [21]:
# get fock properties
C = hf.mo_coeff
h = hf.get_hcore()
J = hf.get_j()
K = hf.get_k()

#print(K.shape)
#print(J.shape)

In [22]:
hf = scf.UHF(mol).run()

print(hf.get_hcore().shape)
print(hf.mo_coeff.shape)

converged SCF energy = -4.71484334984407  <S^2> = 5.0937032e-13  2S+1 = 1
(4, 4)
(2, 4, 4)


In [23]:
# AO MO transformation of elements of fock operator
h1 = expand_matrix(hf.get_hcore())
h1[0:M,0:M] = C[0,:,:].T@(h)@(C[0,:,:])
h1[M:,M:]  = C[1,:,:].T@(h)@(C[1,:,:])
h2 = expand_matrix(hf.get_hcore())
h2[0:M,0:M] = C[0,:,:].T@(J[0,:,:])@(C[0,:,:])
h2[M:,M:]  = C[1,:,:].T@(K[0,:,:])@(C[1,:,:])

#h2 = ao2mo.kernel(mol, hf.mo_coeff)
#https://pyscf.org/_modules/pyscf/fci/direct_uhf.html

In [24]:
e, rdms1 = tools.e_rdms(h1, h2, mol.energy_nuc(), mol.nelec, h1.shape[1],  nroots=3) # i presume this gives 1rdms in the basis set of fock orbitals


RuntimeError: eri.size = 64, norb = 8

In [ ]:
## LFT of Levy Valone

eq_cons = {'type': 'eq',
           'fun' : lambda x: x[0]-h1[0, 0]}
# initializing the calculator
calculator = Calculator(h2, rdms1[0].copy(), # two body term and rdm for the ground state
                        tools.e_rdms, tools.optimize, 1e-7, # function that calculates (energies, rdms) and optimizer 
                        ecore=mol.energy_nuc(), norbs=h1.shape[1], nelec=mol.nelec, maximize=True, symmetrize=True, nroots=5) # parameters for the

# get the exact value of the objective function
L, dL = calculator.calculate_objctv_grdnt(h1)
print("Exact value of the objective function", L)

# run the optimizer
## objectiva function value changes when we add constraint
res = calculator.optimize(np.zeros(h1.size), 
                          method='trust-constr',
                          options={"maxiter":500, 'disp':False, 'verbose':0, 'gtol':1e-7,},
                          constraints=eq_cons)

print("Print value after optimization: ", res.fun)
print("Difference:", res.fun-L)

In [ ]:
FCIoccu, FCInaturalC = np.linalg.eigh(rdms1[0])
print(FCIoccu)